

<br>
<p style="text-align: left;"><img src='https://s3.amazonaws.com/weclouddata/images/logos/sunlife_logo.png' width='35%'></p>
<p style="text-align:left;"><font size='15'><b> Pandas DataFrame II - Intermediate </b></font> <br><font color='#559E54' size=6>Instructor Copy</font></p>
<h2 align='left' > Sunlife Data Science Training </h2>

<h4 align='left'>  Prepared by: <img src='https://s3.amazonaws.com/weclouddata/images/logos/wcd_logo.png' width='15%'></h4>

---


> `pandas` is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python. Additionally, it has the broader goal of becoming the most powerful and flexible open source data analysis / manipulation tool available in any language. It is already well on its way toward this goal.

> `pandas` is well suited for many different kinds of data:

> * Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
* Ordered and unordered (not necessarily fixed-frequency) time series data.
* Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels
* Any other form of observational / statistical data sets. The data actually need not be labeled at all to be placed into a pandas data structure

<h2> References </h2>

* Pandas Reference Guide https://pandas.pydata.org/pandas-docs/stable/

# <a name="toc-8"></a> Table of contents 

1. [Pandas Universal Functions](#ufunc)
2. [Missing Values](#missing)
3. [MultiIndex](#multiinddex)
4. [Concatenation](#concat)
5. [Merge/Join](#merge)

---

# <a name="ufunc"></a><font color='#347B98'> 1. Operating on Data in Pandas - `Ufuncs` </font> <font size='3'>[Back to TOC](#toc-8)</font>


One of the essential pieces of NumPy is the ability to perform quick element-wise operations, both with basic arithmetic (addition, subtraction, multiplication, etc.) and with more sophisticated operations (trigonometric functions, exponential and logarithmic functions, etc.). Pandas inherits much of this functionality from NumPy `Universal Functions`.

In [1]:
import pandas as pd
import numpy as np

## $\Delta$ 1.1 - Index Preservation `Ufuncs`

> Pandas `ufuncs` will preserve **index and column labels** in the output, and for binary operations such as **addition and multiplication**, `Pandas will automatically align indices when passing the objects to the ufunc`. 

In [2]:
random = np.random.RandomState(500)
ser = pd.Series(random.randint(0, 10, 4))
ser

0    7
1    1
2    1
3    8
dtype: int64

In [3]:
df = pd.DataFrame(random.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,7,1,1,5
1,9,2,2,3
2,6,8,7,8


In [4]:
np.exp(ser)

0    1096.633158
1       2.718282
2       2.718282
3    2980.957987
dtype: float64

In [5]:
np.exp(df)

,A,B,C,D
0,1096.633158,2.718282,2.718282,148.413159
1,8103.083928,7.389056,7.389056,20.085537
2,403.428793,2980.957987,1096.633158,2980.957987


In [6]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-0.707107,7.071068e-01,0.707107,-7.071068e-01
1,0.707107,1.000000e+00,1.000000,7.071068e-01
2,-1.000000,-2.449294e-16,-0.707107,-2.449294e-16


## $\Delta$ 1.2 - Index Alignment `Ufuncs`


For binary operations on two Series or DataFrame objects, Pandas will align indices in the process of performing the operation. This is very convenient when working with incomplete data

### Index Alignment in Series

In [7]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')


In [8]:
population / area


Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

The resulting array contains the union of indices of the two input arrays, which could be determined using standard Python set arithmetic on these indices:

In [9]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Any item for which one or the other does not have an entry is marked with NaN, or "Not a Number," which is how Pandas marks missing data. This index matching is implemented this way for any of Python's built-in arithmetic expressions; any missing values are filled in with NaN by default:

In [10]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

If using NaN values is not the desired behavior, calling `A.add(B)` is equivalent to calling `A + B`, but allows optional explicit specification of the fill value for any elements in A or B that might be missing:

In [11]:
A.add(B, fill_value=0)


0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Index Alignment in DataFrame

In [12]:
random = np.random.RandomState(500)
A = pd.DataFrame(random.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,1,17
1,14,8


In [13]:
B = pd.DataFrame(random.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,7,1,1
1,5,9,2
2,2,3,6


In [8]:
A + B

,A,B,C
0,2.0,24.0,NaN
1,23.0,13.0,NaN
2,NaN,NaN,NaN


In [14]:
fill = A.stack().mean()

In [15]:
fill

10.0

In [35]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,2.0,24.0,11.0
1,23.0,13.0,12.0
2,13.0,12.0,16.0


### <center> Pandas Operators </center>  
|Python Operator | Pandas Method(s) |
|---|---|
|+|add()|
|-|sub(), subtract()|
|*|mul(), multiply()|
|/|truediv(), div(), divide()|
|//|floordiv()|
|%|mod()|
|**|pow()|

## $\Delta$ 1.3 - `Unfuncs` - Operations between DataFrame and Series

When performing operations between a DataFrame and a Series, the index and column alignment is similarly maintained. Operations between a DataFrame and a Series are similar to operations between a two-dimensional and one-dimensional NumPy array.

### Row-wise Operations

In [18]:
A = np.random.randint(10, size=(3, 4))
A

array([[9, 1, 4, 9],
       [8, 9, 6, 7],
       [9, 8, 4, 4]])

In [19]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1,  8,  2, -2],
       [ 0,  7,  0, -5]])

#### Row-wise operations in DataFrame

In [20]:
df = pd.DataFrame(A, columns=list('QRST'))
df

,Q,R,S,T
0,9,1,4,9
1,8,9,6,7
2,9,8,4,4


In [21]:
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,8,2,-2
2,0,7,0,-5


### Column-wise Operations
If you would instead like to operate column-wise, you can use the object methods mentioned earlier, while specifying the axis keyword:

In [47]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-1,0,-6,-3
1,6,0,-2,2
2,-3,0,6,-3


In [48]:
df

,Q,R,S,T
0,5,6,0,3
1,9,3,1,5
2,0,3,9,0


In [49]:
halfrow = df.iloc[0, ::2]
halfrow

Q    5
S    0
Name: 0, dtype: int64

In [50]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,4.0,NaN,1.0,NaN
2,-5.0,NaN,9.0,NaN


<br>

# <a name="missing"></a><font color='#347B98'> 2. Handling Missing in Pandas </font> <font size='3'>[Back to TOC](#toc-8)</font>


### Trade-Offs in Missing Data Conventions
There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. Generally, they revolve around one of two strategies: 
* using a `mask` that globally indicates missing values,   
* or choosing a sentinel value that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

### Missing Data in Pandas
The way in which Pandas handles missing values is constrained by its reliance on the `NumPy` package, which does not have a built-in notion of NA values for non-floating-point data types.

NumPy does have support for masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad." Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

**`Pandas chose to use sentinels for missing data`**, and further chose to use two already-existing Python null values: 
> * floating-point **`NaN`** value,   
> * and the Python `None`** object. 



## $\Delta$ 2.1 `None`: Pythonic missing data
The first sentinel value used by Pandas is `None`, a Python singleton object that is often used for missing data in Python code. Because it is a Python object, `None` cannot be used in any arbitrary NumPy/Pandas array, but `only in arrays with data type 'object'` (i.e., arrays of Python objects):

In [23]:
import numpy as np
import pandas as pd

In [24]:
arr1 = np.array([1,None,3,4])
arr1

array([1, None, 3, 4], dtype=object)

> This `dtype=object` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects. While this kind of object array is useful for some purposes, `any operations on the data will be done at the Python level`, with much more overhead than the typically fast operations seen for arrays with native types:

In [25]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
83 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.68 ms ± 6.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



The use of Python objects in an array also means that if you perform aggregations like `sum()` or `min()` across an array with a `None` value, you will generally get an error:

In [58]:
arr1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## $\Delta$ 2.2 Missing Numerical Data

The other missing data representation, `NaN` (acronym for Not a Number), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [26]:
arr2 = np.array([1, np.nan, 3, 4]) 
arr2.dtype

dtype('float64')

In [27]:
arr2

array([ 1., nan,  3.,  4.])

In [28]:
1 + np.nan

nan

In [31]:
arr2.sum(), arr2.min(), arr2.max()

(nan, nan, nan)

In [32]:
np.nansum(arr2), np.nanmin(arr2), np.nanmax(arr2)

(8.0, 1.0, 4.0)

NumPy does provide some special aggregations that will ignore these missing values:

## $\Delta$ 2.3 `None` vs `Nan` in Pandas

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to np.nan, it will automatically be upcast to a floating-point type to accommodate the NA


In [33]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [34]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

For types that don't have an available sentinel value, Pandas automatically type-casts when `NA` values are present. For example, if we set a value in an integer array to `np.nan`, it will automatically be upcast to a floating-point type to accommodate the NA:

In [35]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the None to a NaN value.

The following table lists the upcasting conventions in Pandas when NA values are introduced:


|Typeclass | Pandas Method(s) When Storing NAs | NA Sentinel Value
|---|---|
|floating	|No change	|np.nan|
|object	|No change	|None or np.nan|
|integer	|Cast to float64	|np.nan|
|boolean	|Cast to object	|None or np.nan|

## $\Delta$ 2.4 Operating on `null` values

As we have seen, Pandas treats None and NaN as essentially interchangeable for indicating missing or null values. To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures. They are:

* `isnull()`: Generate a boolean mask indicating missing values
* `notnull()`: Opposite of isnull()
* `dropna()`: Return a filtered version of the data
* `fillna()`: Return a copy of the data with missing values filled or imputed


### Detecting `null` values

Pandas data structures have two useful methods for detecting null data: `isnull()` and `notnull()`. Either one will return a Boolean mask over the data.

In [36]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [37]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [38]:
data[data.notnull()]


0        1
2    hello
dtype: object

### Dropping `null` values

In addition to the masking used before, there are the convenience methods, `dropna()` (which removes NA values) and `fillna()` (which fills in NA values). 

In [39]:
data.dropna()

0        1
2    hello
dtype: object

### Dropping `null` values in DataFrame
We cannot drop single values from a DataFrame; we can only drop full rows or full columns. Depending on the application, you might want one or the other, so dropna() gives a number of options for a DataFrame.

> By default, `dropna()` will drop all rows in which any null value is present

In [40]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [41]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternatively, you can drop NA values along a different axis; axis=1 drops all columns containing a null value:

In [42]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


But this drops some good data as well; you might rather be interested in dropping rows or columns with all NA values, or a majority of NA values. This can be specified through the how or thresh parameters, which allow fine control of the number of nulls to allow through.

In [43]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [44]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [45]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### Filling `null` values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value. This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values. You could do this in-place using the `isnull()` method as a mask, but because it is such a common operation Pandas provides the `fillna()` method, which returns a copy of the array with the null values replaced.

In [46]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [47]:
data.fillna(0)


a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [48]:
# Forward fill
data.fillna(method='ffill')


a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [49]:
# Back fill
data.fillna(method='bfill')


a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [50]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [51]:
df.fillna(method='ffill', axis=1)

,0,1,2
0,1.0,1.0,2.0
1,2.0,3.0,5.0
2,NaN,4.0,6.0


<br>

# <a name="multiinddex"></a><font color='#347B98'> 3. Hierarchical (Multi)Indexing </font> <font size='3'>[Back to TOC](#toc-8)</font>


In [52]:
import pandas as pd
import numpy as np

## $\Delta$ 3.1 Multi-indexed Pandas Series

### Tuple-based Indexing in Series

In [53]:
import numpy as np
import pandas as pd
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [54]:
pop.index

Index([('California', 2000), ('California', 2010),   ('New York', 2000),
         ('New York', 2010),      ('Texas', 2000),      ('Texas', 2010)],
      dtype='object')

In [55]:
pop.values

array([33871648, 37253956, 18976457, 19378102, 20851820, 25145561])

### Slicing a tuple-based index in Series

In [56]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

### Filtering on a tuple-based index in Series

In [57]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

## $\Delta$ 3.2 Introducing Pandas MultiIndex

The tuple-based indexing is essentially a rudimentary multi-index, and the Pandas MultiIndex type gives us the type of operations we wish to have. 

In [58]:
t_index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
index = pd.MultiIndex.from_tuples(t_index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

If we re-index our series with this MultiIndex, we see the hierarchical representation of the data:

In [59]:
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

pop = pd.Series(populations, index=index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

**Question**: 
    
Access all data for which the second index is 2010, we can simply use the Pandas slicing notation

In [60]:
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [108]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### $\delta$ 3.2.1 Conversion between MultiIndex Series and DataFrame using `unstack()` & `stack()`

In [61]:
type(pop)

pandas.core.series.Series

In [62]:
pop.index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [8]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [64]:
pop_df = pop.unstack(level=0)
pop_df

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [65]:
pop_df = pop.unstack(level=1)
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [66]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### $\delta$ 3.2.2 Beyond 2 Dimensions

Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. 

In [67]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [68]:
f_u18 = pop_df['under18'] / pop_df['total']

In [69]:
type(f_u18)

pandas.core.series.Series

In [70]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### $\delta$ 3.2.3 MultiIndex Level Names 

In [71]:
t_index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]

index = pd.MultiIndex.from_tuples(t_index)

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

pop = pd.Series(populations, index=index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [72]:
pop.index 

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [73]:
pop.index.names

FrozenList([None, None])

In [74]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### $\delta$ 3.2.4 MultiIndex for Columns

In [75]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  37.1  15.0  37.8  29.0  37.4
     2      37.0  36.9  34.0  37.7  53.0  36.4
2014 1      37.0  36.8  35.0  36.7  27.0  37.3
     2      30.0  36.7  26.0  36.2  40.0  36.7

In [77]:
health_data['Sue']


type          HR  Temp
year visit            
2013 1      29.0  37.4
     2      53.0  36.4
2014 1      27.0  37.3
     2      40.0  36.7

### $\delta$ 3.2.5 Slicing and Indexing on MultiIndexed Series

In [78]:
t_index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]

index = pd.MultiIndex.from_tuples(t_index)

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

pop = pd.Series(populations, index=index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:

In [79]:
pop['California', 2000]

33871648

In [80]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

#### Partial Slicing

In [81]:
pop.loc['California':'New York']

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

#### Partial Indexing

In [82]:
pop[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

#### Boolean Mask

In [83]:
pop[pop > 22000000]

California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

#### Fancy Indexing

In [84]:
pop[['California', 'Texas']]

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### $\delta$ 3.2.6 Slicing and Indexing on MultiIndexed DataFrames

In [85]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  37.6  42.0  36.5  27.0  38.1
     2      47.0  37.8  24.0  37.4  33.0  35.8
2014 1      21.0  35.8  41.0  37.3  49.0  37.8
     2      45.0  36.5  27.0  37.7  24.0  35.0

In [86]:
health_data['Guido', 'HR']

year  visit
2013  1        42.0
      2        24.0
2014  1        41.0
      2        27.0
Name: (Guido, HR), dtype: float64

In [87]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      43.0  37.6
     2      47.0  37.8

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in loc or iloc can be passed a tuple of multiple indices

In [88]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        43.0
      2        47.0
2014  1        21.0
      2        45.0
Name: (Bob, HR), dtype: float64

In [89]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-89-fb34fa30ac09>, line 1)

In [90]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,43.0,42.0,27.0
2014,1,21.0,41.0,49.0


### $\delta$ 3.2.7 Stacking and Unstacking MultiIndex

In [91]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [92]:
pop.index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [93]:
pop.unstack(level=0)

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [94]:
pop.unstack(level=1)

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [95]:
pop.unstack().stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### $\delta$ 3.2.8 Setting/Resetting Index

In [96]:
pop.index.names = ['state', 'year']

In [97]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [98]:
type(pop_flat)

pandas.core.frame.DataFrame

In [99]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### $\delta$ 3.2.9 Aggregation on MultiIndex

In [100]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  37.6  42.0  36.5  27.0  38.1
     2      47.0  37.8  24.0  37.4  33.0  35.8
2014 1      21.0  35.8  41.0  37.3  49.0  37.8
     2      45.0  36.5  27.0  37.7  24.0  35.0

In [101]:
data_mean = health_data.mean(level='year')

In [102]:
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     45.0  37.70  33.0  36.95  30.0  36.95
2014     33.0  36.15  34.0  37.50  36.5  36.40

In [103]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,36.0,37.200000
2014,34.5,36.683333


    
<br>
    
# <a name="concat"></a><font color='#347B98'> 4. Combining Datasets: `Concat` and `Append` </font> <font size='3'>[Back to TOC](#toc-8)</font>


In [104]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


## $\Delta$ 4.1 Concatenate in Numpy

In [105]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [106]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## $\Delta$ 4.2 Pandas Concatenation with `pd.concat`

### $\delta$ 4.2.1 Concatenating Series

In [107]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

### $\delta$ 4.2.2 Concatenating DataFrame

#### Row-wise Concatenation

In [108]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])


In [109]:
df1

,A,B
1,A1,B1
2,A2,B2


In [110]:
df2

,A,B
3,A3,B3
4,A4,B4


In [111]:
pd.concat([df1, df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


#### Column-wise Concatenation

In [112]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])

In [113]:
df3

,A,B
0,A0,B0
1,A1,B1


In [114]:
df4

,C,D
0,C0,D0
1,C1,D1


In [115]:
pd.concat([df3, df4], axis=1)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


#### Concatenating with preserved indices
> Pandas `concat()` preserves indices, which means it will cause duplicated indices

In [116]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!

In [117]:
x

,A,B
0,A0,B0
1,A1,B1


In [118]:
y

,A,B
0,A2,B2
1,A3,B3


In [119]:
pd.concat([x,y])

,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


While duplicate indices is valid within DataFrames, the outcome is often undesirable. `pd.concat()` gives us a few ways to handle it.

In [120]:
# check duplicate index and return error
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


In [121]:
# Ignoring duplicate index
pd.concat([x, y], ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


Another option is to use the keys option to specify a label for the data sources; the result will be a hierarchically indexed series containing the data

In [122]:
pd.concat([x, y], keys=['x', 'y'])

A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

#### Concatenation with joined columns

In the simple examples we just looked at, we were mainly concatenating DataFrames with shared column names. In practice, data from different sources might have different sets of column names, and `pd.concat` offers several options in this case. 

In [123]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])

In [124]:
df5

,A,B,C
1,A1,B1,C1
2,A2,B2,C2


In [125]:
df6

,B,C,D
3,B3,C3,D3
4,B4,C4,D4


In [126]:
pd.concat([df5, df6])

/usr/lib64/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


By default, the entries for which no data is available are filled with `NA` values. To change this, we can specify one of several options for the `join` and `join_axes` parameters of the concatenate function. By default, the join is a union of the input columns (`join='outer'`), but we can change this to an intersection of the columns using `join='inner'`

In [127]:
pd.concat([df5, df6], join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


In [128]:
pd.concat([df5, df6], join_axes=[df5.columns])

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


## $\Delta$ 4.3 Pandas Concatenation with `pd.append`

Unlike the `append()` and `extend()` methods of Python lists, the **`append()` method in Pandas does not modify the original object. Instead it creates a new object with the combined data**. 
> - Therefore, `pd.append` is not a very efficient method, because it involves creation of a new index and data buffer.    
> - Thus, if you plan to do multiple append operations, it is generally better to build a list of DataFrames and pass them all at once to the concat() function.

In [201]:
df1

,A,B
1,A1,B1
2,A2,B2


In [202]:
df2

,A,B
3,A3,B3
4,A4,B4


In [203]:
df1.append(df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [219]:
pd.concat([df1, df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


### $\delta$ 4.3.1 Concatenation: `pd.concat()` vs `pd.append()`

In [217]:
df = pd.DataFrame(np.random.randn(100000,20))

df['B'] = 'foo'
df['C'] = pd.Timestamp('20130101')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
0     100000 non-null float64
1     100000 non-null float64
2     100000 non-null float64
3     100000 non-null float64
4     100000 non-null float64
5     100000 non-null float64
6     100000 non-null float64
7     100000 non-null float64
8     100000 non-null float64
9     100000 non-null float64
10    100000 non-null float64
11    100000 non-null float64
12    100000 non-null float64
13    100000 non-null float64
14    100000 non-null float64
15    100000 non-null float64
16    100000 non-null float64
17    100000 non-null float64
18    100000 non-null float64
19    100000 non-null float64
B     100000 non-null object
C     100000 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(20), object(1)
memory usage: 16.8+ MB


In [220]:
def f1():
    result = df
    for i in range(9):
        result = result.append(df)
    return result

In [223]:
def f2():
    result = []
    for i in range(10):
        result.append(df)
    return pd.concat(result)

In [224]:
f1().equals(f2())

True

In [225]:
%timeit f1()

727 ms ± 71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [226]:
%timeit f2()

141 ms ± 5.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


    
<br>
    
# <a name="merge"></a><font color='#347B98'> 5. Combining Datasets: `Merge` and `Join` </font> <font size='3'>[Back to TOC](#toc-8)</font>


#### Datasets for the `merge` lab

In [129]:
employee = pd.DataFrame({'employee': ['Mark', 'Austin', 'Rob', 'Coco', 'Julian', 'Akanksha', 'Danielle'],
                    'group': ['Loyalty', 'Loyalty', 'Loyalty', 'Loyalty', 'CTFS', 'Assortment', 'Promo Planning']})
hr = pd.DataFrame({'employee': ['Rob', 'Mark', 'Austin','Akanksha','Julian', 'Coco', 'Danielle'],
                    'hire_date': [2017, 2018, 2018, 2017, 2016, 2015, 2015]})
review = pd.DataFrame({'name': ['Rob', 'Mark', 'Austin','Akanksha','Julian', 'Coco', 'Danielle'],
                    'performance': [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]})
supervisor = pd.DataFrame({'group': ['Loyalty', 'CTFS', 'Assortment', 'Promo Planning'],
                    'supervisor': ['Mark', 'Guido', 'Steve', 'Amir']})
skills = pd.DataFrame({'group': ['Loyalty', 'Loyalty','Loyalty',
                              'CTFS', 'CTFS', 'Assortment', 'Assortment', 'Promo Planning', 'Promo Planning'],
                    'skills': ['math', 'tableau', 'spark',
                               'optimization', 'campaign', 'sas', 'matlab', 'modeling', 'r']})

In [130]:
employee

,employee,group
0,Mark,Loyalty
1,Austin,Loyalty
2,Rob,Loyalty
3,Coco,Loyalty
4,Julian,CTFS
5,Akanksha,Assortment
6,Danielle,Promo Planning


In [131]:
hr

,employee,hire_date
0,Rob,2017
1,Mark,2018
2,Austin,2018
3,Akanksha,2017
4,Julian,2016
5,Coco,2015
6,Danielle,2015


In [132]:
review

,name,performance
0,Rob,5.0
1,Mark,5.0
2,Austin,5.0
3,Akanksha,5.0
4,Julian,5.0
5,Coco,5.0
6,Danielle,5.0


In [133]:
supervisor

,group,supervisor
0,Loyalty,Mark
1,CTFS,Guido
2,Assortment,Steve
3,Promo Planning,Amir


In [134]:
skills

,group,skills
0,Loyalty,math
1,Loyalty,tableau
2,Loyalty,spark
3,CTFS,optimization
4,CTFS,campaign
5,Assortment,sas
6,Assortment,matlab
7,Promo Planning,modeling
8,Promo Planning,r


## $\Delta$ 5.1 Merging Datasets

### $\delta$ 5.1.1 One-to-One Joins
The simplest type of merge expresion is the one-to-one join, which is in many ways very similar to the column-wise concatenation 

#### Question: Join employee dataframe and hr dataframe to create a df that looks like the following
|employee | group | hire_date|
|---|---|---|
|Mark	|Loyalty	|2018|
|Austin	|Loyalty|	2018|
|Rob	|Loyalty	|2017|
| ... | ... | ... |

In [135]:
employee_1 = pd.merge(employee, hr)

In [61]:
employee_1

,employee,group,hire_date
0,Mark,Loyalty,2018
1,Austin,Loyalty,2018
2,Rob,Loyalty,2017
3,Coco,Loyalty,2015
4,Julian,CTFS,2016
5,Akanksha,Assortment,2017
6,Danielle,Promo Planning,2015


#### Question: Try to create the same table using `pd.concat` column-wise join
> Make sure that columns with duplicate names are removed

In [62]:
#########################
# Your Code Below
#########################

In [27]:
pd.concat([employee, hr], axis=1)

,employee,group,employee,hire_date
0,Mark,Loyalty,Rob,2017
1,Austin,Loyalty,Mark,2018
2,Rob,Loyalty,Austin,2018
3,Coco,Loyalty,Akanksha,2017
4,Julian,CTFS,Julian,2016
5,Akanksha,Assortment,Coco,2015
6,Danielle,Promo Planning,Danielle,2015


In [29]:
employee.sort_values(['employee'], inplace=True)
employee_1 = employee.set_index('employee')
employee_1

,group
employee,
Akanksha,Assortment
Austin,Loyalty
Coco,Loyalty
Danielle,Promo Planning
Julian,CTFS
Mark,Loyalty
Rob,Loyalty


In [30]:
hr.sort_values(['employee'], inplace=True)
hr_1 = hr.set_index('employee')
hr_1

,hire_date
employee,
Akanksha,2017
Austin,2018
Coco,2015
Danielle,2015
Julian,2016
Mark,2018
Rob,2017


In [25]:
employee_1_ = pd.concat([employee_1, hr_1], axis=1, join='inner')
employee_1_

,group,hire_date
employee,,
Akanksha,Assortment,2017
Austin,Loyalty,2018
Coco,Loyalty,2015
Danielle,Promo Planning,2015
Julian,CTFS,2016
Mark,Loyalty,2018
Rob,Loyalty,2017


In [273]:
employee_1_.drop(hr_1.columns[0], axis=1)

,group,hire_date
0,Loyalty,2017
1,Loyalty,2018
2,Loyalty,2018
3,Loyalty,2017
4,CTFS,2016
5,Assortment,2015
6,Promo Planning,2015


In [69]:
employee_1_

,employee,group,employee,hire_date
0,Mark,Loyalty,Rob,2017
1,Austin,Loyalty,Mark,2018
2,Rob,Loyalty,Austin,2018
3,Coco,Loyalty,Akanksha,2017
4,Julian,CTFS,Julian,2016
5,Akanksha,Assortment,Coco,2015
6,Danielle,Promo Planning,Danielle,2015


In [70]:
employee_1_.iloc[:, [0, 1, 3]]

,employee,group,hire_date
0,Mark,Loyalty,2017
1,Austin,Loyalty,2018
2,Rob,Loyalty,2018
3,Coco,Loyalty,2017
4,Julian,CTFS,2016
5,Akanksha,Assortment,2015
6,Danielle,Promo Planning,2015


In [274]:
employee_1_.iloc[:, [ i for i, j in enumerate(employee_1_.columns) if i!=2 ]]

,employee,group,hire_date
0,Mark,Loyalty,2017
1,Austin,Loyalty,2018
2,Rob,Loyalty,2018
3,Coco,Loyalty,2017
4,Julian,CTFS,2016
5,Akanksha,Assortment,2015
6,Danielle,Promo Planning,2015


### $\delta$ 5.1.2 Many-to-One Join
Many-to-one joins are joins in which one of the two key columns contains duplicate entries. For the many-to-one case, the resulting DataFrame will preserve those duplicate entries as appropriate. 



#### Question - Create a table that joins employee_1 and supervisor
|employee | group | hire_date| supervisor |
|---|---|---|---|
|employee|group|hire_date|supervisor|
|Mark|Loyalty|2018|Mark|
|Austin|Loyalty|2018|Mark|
|Rob|Loyalty|2017|Mark|

In [71]:
employee_1.columns

Index(['employee', 'group', 'hire_date'], dtype='object')

In [72]:
supervisor.columns

Index(['group', 'supervisor'], dtype='object')

In [73]:
supervisor

,group,supervisor
0,Loyalty,Mark
1,CTFS,Guido
2,Assortment,Steve
3,Promo Planning,Amir


In [278]:
##########################
# YOUR CODE BELOW
##########################
employee_2 = pd.merge(employee_1, supervisor)
employee_2

,employee,group,hire_date,supervisor
0,Mark,Loyalty,2018,Mark
1,Austin,Loyalty,2018,Mark
2,Rob,Loyalty,2017,Mark
3,Coco,Loyalty,2015,Mark
4,Julian,CTFS,2016,Guido
5,Akanksha,Assortment,2017,Steve
6,Danielle,Promo Planning,2015,Amir


### $\delta$ 5.1.3 Many-to-Many Join

#### Question - Create a table like shown below

|employee | group | hire_date| supervisor | skills |
|---|---|---|---|---|
|Mark	|Loyalty	|2018	|Mark	|math|
|Mark	|Loyalty	|2018	|Mark	|tableau|
|Mark	|Loyalty	|2018	|Mark	|spark|
|Austin	|Loyalty	|2018	|Mark	|math|
|Austin	|Loyalty	|2018	|Mark	|tableau|
|Austin	|Loyalty	|2018	|Mark	|spark|

In [281]:
##########################
# YOUR CODE BELOW
##########################
employee_3 = pd.merge(employee_2, skills)
employee_3

,employee,group,hire_date,supervisor,skills
0,Mark,Loyalty,2018,Mark,math
1,Mark,Loyalty,2018,Mark,tableau
2,Mark,Loyalty,2018,Mark,spark
3,Austin,Loyalty,2018,Mark,math
4,Austin,Loyalty,2018,Mark,tableau
5,Austin,Loyalty,2018,Mark,spark
6,Rob,Loyalty,2017,Mark,math
7,Rob,Loyalty,2017,Mark,tableau
8,Rob,Loyalty,2017,Mark,spark
9,Coco,Loyalty,2015,Mark,math


### $\delta$ 5.1.4 Merge by `on` key
Pandas `pd.merge` looks for **`one or more matching column names between the two inputs, and uses this as the key`**. However, often the column names will not match so nicely, and `pd.merge()` provides a variety of options for handling this.

In [283]:
pd.merge(employee, hr, on='employee')

,employee,group,hire_date
0,Mark,Loyalty,2018
1,Austin,Loyalty,2018
2,Rob,Loyalty,2017
3,Coco,Loyalty,2015
4,Julian,CTFS,2016
5,Akanksha,Assortment,2017
6,Danielle,Promo Planning,2015


### $\delta$ 5.1.5 Merge by `left_on|right_on` key

At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as "name" rather than "employee". 

In [74]:
review

,name,performance
0,Rob,5.0
1,Mark,5.0
2,Austin,5.0
3,Akanksha,5.0
4,Julian,5.0
5,Coco,5.0
6,Danielle,5.0


In [285]:
pd.merge(employee, review, left_on='employee', right_on='name')

,employee,group,name,performance
0,Mark,Loyalty,Mark,5.0
1,Austin,Loyalty,Austin,5.0
2,Rob,Loyalty,Rob,5.0
3,Coco,Loyalty,Coco,5.0
4,Julian,CTFS,Julian,5.0
5,Akanksha,Assortment,Akanksha,5.0
6,Danielle,Promo Planning,Danielle,5.0


### $\delta$ 5.1.5 Merge by `left_index|right_index`

#### <font color='#FC7307'> Both tables have index set

In [78]:
employee

,employee,group
0,Mark,Loyalty
1,Austin,Loyalty
2,Rob,Loyalty
3,Coco,Loyalty
4,Julian,CTFS
5,Akanksha,Assortment
6,Danielle,Promo Planning


In [75]:
employee_ = employee.set_index('employee')
hr_ = hr.set_index('employee')

In [76]:
employee_

,group
employee,
Mark,Loyalty
Austin,Loyalty
Rob,Loyalty
Coco,Loyalty
Julian,CTFS
Akanksha,Assortment
Danielle,Promo Planning


In [77]:
hr_

,hire_date
employee,
Rob,2017
Mark,2018
Austin,2018
Akanksha,2017
Julian,2016
Coco,2015
Danielle,2015


In [306]:
pd.merge(employee_, hr_, left_index=True, right_index=True)

,group,hire_date
employee,,
Mark,Loyalty,2018
Austin,Loyalty,2018
Rob,Loyalty,2017
Coco,Loyalty,2015
Julian,CTFS,2016
Akanksha,Assortment,2017
Danielle,Promo Planning,2015


#### <font color='#FC7307'> One side has index set

In [79]:
employee_ = employee
hr_ = hr.set_index('employee')

In [80]:
employee_

,employee,group
0,Mark,Loyalty
1,Austin,Loyalty
2,Rob,Loyalty
3,Coco,Loyalty
4,Julian,CTFS
5,Akanksha,Assortment
6,Danielle,Promo Planning


In [81]:
hr_

,hire_date
employee,
Rob,2017
Mark,2018
Austin,2018
Akanksha,2017
Julian,2016
Coco,2015
Danielle,2015


In [318]:
pd.merge(employee_, hr_, left_on='employee', right_index=True)

,employee,group,hire_date
0,Mark,Loyalty,2018
1,Austin,Loyalty,2018
2,Rob,Loyalty,2017
3,Coco,Loyalty,2015
4,Julian,CTFS,2016
5,Akanksha,Assortment,2017
6,Danielle,Promo Planning,2015


### $\delta$ <font color='#559E54'> 5.1 Dataset Merge Labs - Join Multiple Tables
    

#### Lab Question 1 - Join multiple tables in one statement and create the following table
* Hint: `pd.merge` == `df1.merge(df2)`

<br>
<center> Expected Output:  </center>

|employee | group | hire_date | performance |
|---|---|---|---|
|Mark|Loyalty|2018|5.0|
|Austin|Loyalty|2018|5.0|
|Rob|Loyalty|2017|5.0|


|employee | group | hire_date | performance |
|---|---|---|---|---|
|employee|group|hire_date|performance|
|Mark|Loyalty|2018|5.0|
|Austin|Loyalty|2018|5.0|
|Rob|Loyalty|2017|5.0|

In [289]:
########################
# Your Code Below
########################

(employee.merge(hr, on='employee')
         .merge(review, left_on='employee', right_on='name')
         .drop('name', axis=1)
)

,employee,group,hire_date,performance
0,Mark,Loyalty,2018,5.0
1,Austin,Loyalty,2018,5.0
2,Rob,Loyalty,2017,5.0
3,Coco,Loyalty,2015,5.0
4,Julian,CTFS,2016,5.0
5,Akanksha,Assortment,2017,5.0
6,Danielle,Promo Planning,2015,5.0


#### Lab Question 2 - Use `functools.reduce` to join multiple tables with same common join keys
<br>
<center> Expected Output:  </center>

|employee | group | hire_date | performance |
|---|---|---|---|
|Mark|	Loyalty|	2018	|	5.0|
|Austin|	Loyalty	|2018|		5.0|
|Rob	|Loyalty	|2017	|	5.0|

In [291]:
employee = pd.DataFrame({'employee': ['Mark', 'Austin', 'Rob', 'Coco', 'Julian', 'Akanksha', 'Danielle'],
                    'group': ['Loyalty', 'Loyalty', 'Loyalty', 'Loyalty', 'CTFS', 'Assortment', 'Promo Planning']})
hr = pd.DataFrame({'employee': ['Rob', 'Mark', 'Austin','Akanksha','Julian', 'Coco', 'Danielle'],
                    'hire_date': [2017, 2018, 2018, 2017, 2016, 2015, 2015]})
review = pd.DataFrame({'employee': ['Rob', 'Mark', 'Austin','Akanksha','Julian', 'Coco', 'Danielle'],
                    'performance': [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]})

In [293]:
########################
# Your Code Below
########################
from functools import reduce

dfs = [employee, hr, review]

df_final = reduce(lambda left,right: pd.merge(left,right,on='employee'), dfs)

In [294]:
df_final

,employee,group,hire_date,performance
0,Mark,Loyalty,2018,5.0
1,Austin,Loyalty,2018,5.0
2,Rob,Loyalty,2017,5.0
3,Coco,Loyalty,2015,5.0
4,Julian,CTFS,2016,5.0
5,Akanksha,Assortment,2017,5.0
6,Danielle,Promo Planning,2015,5.0


---

## $\Delta$ 5.2 Joining Datasets

#### Datasets for the `Join` lab

In [31]:
employee = pd.DataFrame({'employee': ['Mark', 'Austin', 'Rob', 'Coco', 'Julian', 'Akanksha', 'Danielle'],
                    'group': ['Loyalty', 'Loyalty', 'Loyalty', 'Loyalty', 'CTFS', 'Assortment', 'Promo Planning']})
hr = pd.DataFrame({'employee': ['Rob', 'Mark', 'Austin','Akanksha','Julian', 'Coco', 'Danielle'],
                    'year': [2017, 2018, 2016, 2017, 2016, 2015, 2015]})
promotion = pd.DataFrame({'employee': ['Austin','Akanksha','Julian','Danielle'],
                    'year': [2018, 2017, 2017, 2016]})
review = pd.DataFrame({'name': ['Rob', 'Mark', 'Austin','Akanksha','Julian', 'Coco', 'Danielle'],
                    'performance': [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]})
supervisor = pd.DataFrame({'group': ['Loyalty', 'CTFS', 'Assortment', 'Promo Planning'],
                    'supervisor': ['Mark', 'Guido', 'Steve', 'Amir']})
skills = pd.DataFrame({'group': ['Loyalty', 'Loyalty','Loyalty',
                              'CTFS', 'CTFS', 'Assortment', 'Assortment', 'Promo Planning', 'Promo Planning'],
                    'skills': ['math', 'tableau', 'spark',
                               'optimization', 'campaign', 'sas', 'matlab', 'modeling', 'r']})
education = pd.DataFrame({'employee': ['Austin', 'Coco', 'Danielle'],
                    'education': ['Master', 'Master', 'PhD']})
hobby = pd.DataFrame({'employee': ['Austin', 'Owen', 'Julian', 'Bob'],
                    'hobby': ['soccer', 'jogging', 'AI', 'hockey']})

In [351]:
employee

,employee,group
0,Mark,Loyalty
1,Austin,Loyalty
2,Rob,Loyalty
3,Coco,Loyalty
4,Julian,CTFS
5,Akanksha,Assortment
6,Danielle,Promo Planning


In [352]:
hobby

,employee,hobby
0,Austin,soccer
1,Owen,jogging
2,Julian,AI
3,Bob,hockey


### $\delta$ 5.2.1 `Inner` Join
> `pd.merge` by default implements `inner join`

In [353]:
pd.merge(employee, hobby)

,employee,group,hobby
0,Austin,Loyalty,soccer
1,Julian,CTFS,AI


In [354]:
pd.merge(employee, hobby, how='inner')

,employee,group,hobby
0,Austin,Loyalty,soccer
1,Julian,CTFS,AI


### $\delta$ 5.2.2 `Left` Join

In [355]:
pd.merge(employee, hobby, how='left')

,employee,group,hobby
0,Mark,Loyalty,NaN
1,Austin,Loyalty,soccer
2,Rob,Loyalty,NaN
3,Coco,Loyalty,NaN
4,Julian,CTFS,AI
5,Akanksha,Assortment,NaN
6,Danielle,Promo Planning,NaN


### $\delta$ 5.2.3 `Outer` Join

In [356]:
pd.merge(employee, hobby, how='outer')

,employee,group,hobby
0,Mark,Loyalty,NaN
1,Austin,Loyalty,soccer
2,Rob,Loyalty,NaN
3,Coco,Loyalty,NaN
4,Julian,CTFS,AI
5,Akanksha,Assortment,NaN
6,Danielle,Promo Planning,NaN
7,Owen,NaN,jogging
8,Bob,NaN,hockey


### $\delta$ 5.2.3 Handling Overlapping Column Names - `Suffixes`

In [357]:
hr

,employee,year
0,Rob,2017
1,Mark,2018
2,Austin,2016
3,Akanksha,2017
4,Julian,2016
5,Coco,2015
6,Danielle,2015


In [358]:
promotion

,employee,year
0,Austin,2018
1,Akanksha,2017
2,Julian,2017
3,Danielle,2016


#### For same column names, pandas automatically append suffix

In [359]:
pd.merge(hr, promotion, left_on='employee', right_on='employee', how='outer')

,employee,year_x,year_y
0,Rob,2017,NaN
1,Mark,2018,NaN
2,Austin,2016,2018.0
3,Akanksha,2017,2017.0
4,Julian,2016,2017.0
5,Coco,2015,NaN
6,Danielle,2015,2016.0


#### Customizing suffixes

In [362]:
pd.merge(hr, promotion, left_on='employee', right_on='employee', how='outer', suffixes=['_hire', '_promo'])

,employee,year_hire,year_promo
0,Rob,2017,NaN
1,Mark,2018,NaN
2,Austin,2016,2018.0
3,Akanksha,2017,2017.0
4,Julian,2016,2017.0
5,Coco,2015,NaN
6,Danielle,2015,2016.0
